# Usage

## Using the included categorizations

In the `climate_categories` package, the categorizations are available
directly at the top-level namespace, and as a dictionary in `.cats`:

In [ ]:
import climate_categories

climate_categories.cats

In [ ]:
climate_categories.IPCC2006

In [ ]:
climate_categories.cats["IPCC2006"]

Metadata for each categorization are accessible as properties:

In [ ]:
print(climate_categories.IPCC2006.name)
print(climate_categories.IPCC2006.title)
print(climate_categories.IPCC2006.comment)
print(climate_categories.IPCC2006.references)
print(climate_categories.IPCC2006.institution)
print(climate_categories.IPCC2006.last_update)
print(climate_categories.IPCC2006.version)

The categorization can be used as a dictionary to query the meaning
of category codes:

In [ ]:
climate_categories.IPCC2006["1.A"]

You can also query using alternative spellings of the code:

In [ ]:
climate_categories.IPCC2006["1A"]

For the categories, metadata is also available: a `title`, maybe a
`comment`, all of its `codes` and possibly additional non-standard information
in the `info` dictionary:

In [ ]:
one_a = climate_categories.IPCC2006["1.A"]
print(one_a.title)
print(one_a.comment)
print(one_a.codes)
print(one_a.info)

For hierarchical categorizations, you can also query for parent and child
categories.
Note that a list of sets of children is returned in case a category
can be composed differently:

In [ ]:
climate_categories.IPCC2006["1.A"].children

In [ ]:
climate_categories.IPCC2006["1.A"].parents

Finally, you can check if a categorization is hierarchical, and
for hierarchical categorizations you can check if the sum of all
child categories should be equal to the sum of parent categories:

In [ ]:
print(f"Hierachical: {climate_categories.IPCC2006.hierarchical}")
print(f"Total sum: {climate_categories.IPCC2006.total_sum}")

## Extending categorizations

Often, you want to use a common categorization, but for one reason or
another, you have to add a couple of categories. This is possible:

In [ ]:
IPCC2006_PRIMAP = climate_categories.IPCC2006.extend(
    name="PRIMAP",
    categories={
        "M0": {"title": "TOTAL", "comment": "All emissions and removals"}
    },
    children=[("M0", ("1", "2", "3", "4", "5"))],
)

print(IPCC2006_PRIMAP.name)
print(IPCC2006_PRIMAP.title)
print(IPCC2006_PRIMAP.comment)

If the canonical top level category of hierarchical
categorizations is defined, you can also calculate the level of a category in the
hierarchy:

In [ ]:
IPCC2006_PRIMAP.canonical_top_level_category = IPCC2006_PRIMAP["M0"]

print(IPCC2006_PRIMAP["M0"].level)
print(IPCC2006_PRIMAP["1.A"].level)

## Pandas integration

For each categorization, the categories are also available as a pandas
DataFrame:

In [ ]:
climate_categories.IPCC2006.df